In [1]:
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta
import re
import os,sys

import kbij_helper as kbijh
kbij = kbijh.KBIJ()

import warnings
warnings.filterwarnings('ignore')
autostats_lib = "../../../../artemis/autostats"
if autostats_lib not in sys.path:
    sys.path.append(autostats_lib)

from utilities.data_storage import DataStorage
import time

In [2]:
%load_ext google.cloud.bigquery
from google.cloud import bigquery
DataStorage(os.environ["GOOGLE_APPLICATION_CREDENTIALS"])

from IPython.display import clear_output

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [3]:
def get_key(my_dict, val):
    keys_list = []
    for key, value in my_dict.items(): 
         if val == value: 
             keys_list.append(key)
  
    return keys_list

In [4]:
date = "2020-02-01"
prev_date = "2020-01-01"

In [5]:
datem = pd.to_datetime(date)
datem = datem.strftime("%Y%m")
print(datem)

202002


In [6]:
pathname = 'data'+datem
dirname = './'+pathname

In [7]:
raw_filename_F01 = dirname+'/F01'+datem+'.csv'
raw_filename_D01 = dirname+'/D01'+datem+'.csv'

In [8]:
filtered_filename_F01 = dirname+'/F01'+datem+'_new_filtered.csv'
filtered_filename_D01 = dirname+'/D01'+datem+'_new_filtered.csv'

In [9]:
prev_datem = pd.to_datetime(prev_date)
prev_datem = prev_datem.strftime("%Y%m")
print(prev_datem)

202001


In [10]:
prev_pathname = 'data'+prev_datem
prev_dirname = './'+prev_pathname

In [11]:
prev_filtered_filename_F01 = prev_dirname+'/F01'+prev_datem+'_new_filtered.csv'
prev_filtered_filename_D01 = prev_dirname+'/D01'+prev_datem+'_new_filtered.csv'

In [12]:
F01_dtype = {'Kode Jenis Pelapor': 'object',
 'Kode Pelapor': 'object',
 'Tahun Bulan Data': 'object',
 'Kode Jenis Fasilitas': 'object',
 'Nomor Rekening Fasilitas': 'object',
 'Nomor Rekening Lama Fasilitas': 'object',
 'Nomor CIF Debitur': 'object',
 'Kode Sifat Kredit atau Pembiayaan': 'str',
 'Kode Jenis Kredit atau Pembiayaan': 'str',
 'Kode Akad Kredit atau Akad Pembiayaan': 'object',
 'Nomor Akad Awal': 'object',
 'Tanggal Akad Awal': 'object',
 'Nomor Akad Akhir': 'object',
 'Tanggal Akad Akhir': 'object',
 'Frekuensi Perpanjangan Fasilitas Kredit atau Pembiayaan': 'int64',
 'Tanggal Awal Kredit atau Pembiayaan': 'object',
 'Tanggal Mulai': 'object',
 'Tanggal Jatuh Tempo': 'object',
 'Kode Kategori Debitur': 'object',
 'Kode Jenis Penggunaan': 'str',
 'Kode Orientasi Penggunaan': 'str',
 'Kode Sektor Ekonomi': 'object',
 'Kode Kabupaten atau Kota Lokasi Proyek atau Penggunaan Kredit atau Pembiayaan': 'object',
 'Nilai Proyek': 'object',
 'Kode Valuta': 'object',
 'Suku Bunga atau Imbalan': 'float64',
 'Jenis Suku Bunga atau Imbalan': 'int64',
 'Kredit atau Pembiayaan Program Pemerintah': 'object',
 'Plafon Awal': 'float64',
 'Plafon': 'float64',
 'Realisasi atau Pencairan Bulan Berjalan': 'float64',
 'Denda': 'float64',
 'Baki Debet': 'float64',
 'Nilai dalam Mata Uang Asal': 'object',
 'Kode Kualitas Kredit atau Pembiayaan': 'str',
 'Tanggal Macet': 'object',
 'Kode Sebab Macet': 'str',
 'Tunggakan Pokok': 'int64',
 'Tunggakan Bunga atau Imbalan': 'int64',
 'Jumlah Hari Tunggakan': 'int64',
 'Frekuensi Tunggakan': 'int64',
 'Frekuensi Restrukturisasi': 'int64',
 'Tanggal Restrukturisasi Awal': 'object',
 'Tanggal Restrukturisasi Akhir': 'object',
 'Kode Cara Restrukturisasi': 'object',
 'Kode Kondisi': 'object',
 'Tanggal Kondisi': 'object',
 'Keterangan': 'object',
 'Kode Kantor Cabang': 'object',
 'Operasi Data': 'object',
 'Status delete': 'object',
 'Create Date': 'str',
 'Update Date': 'str',
 'Approved Date': 'str',
 'userId': 'object',
 'orderId': 'object',
 'status': 'object'}

D01_dtype = {'Kode Jenis Pelapor': 'object',
 'Kode Pelapor': 'object',
 'Tahun Bulan Data': 'object',
 'Nomor CIF Lama Debitur': 'object',
 'Nomor CIF Debitur': 'object',
 'Jenis Identitas': 'int64',
 'Nomor Identitas': 'object',
 'Nama Sesuai Identitas': 'object',
 'Nama Lengkap': 'object',
 'Kode Status Pendidikan atau Gelar Debitur': 'object',
 'Jenis Kelamin': 'object',
 'Tempat Lahir': 'object',
 'Tanggal Lahir': 'object',
 'Nama Gadis Ibu Kandung': 'object',
 'NPWP': 'object',
 'Alamat': 'object',
 'Kelurahan': 'object',
 'Kecamatan': 'object',
 'Kode Kabupaten atau Kota': 'object',
 'Kode Pos': 'object',
 'Nomor Telepon': 'object',
 'Nomor Telepon Seluler': 'object',
 'Alamat email': 'object',
 'Kode Negara Domisili': 'object',
 'Kode Pekerjaan': 'object',
 'Tempat Bekerja': 'object',
 'Kode Bidang Usaha Tempat Bekerja': 'object',
 'Alamat Tempat Bekerja': 'object',
 'Kode Golongan Debitur': 'object',
 'Status Perkawinan Debitur': 'int64',
 'Kode Kantor Cabang': 'object',
 'Operasi Data': 'object',
 'Status delete': 'object',
 'Create Date': 'str',
 'Update Date': 'str',
 'Approved Date': 'str',
 'userId': 'object',
 'orderId': 'object',
 'status': 'object'}

Data Preparation Process for D01 and F01 KBIJ =
1. Import previous data 
2. Import full data dump from query 
3. Find Delta between new data dump and previous data 
4. Export delta to csv and send to KBIJ 
5. Export full data dump to csv to be used in the next iteration

# Query

Get new F01 and D01 from redash

## F01 Current Data

In [5]:
F01new = kbij.get_F01_data('"'+date+'"')

[get_F01_data] Query from BigQuery takes 75.25130724906921


## D01 Current Data

In [6]:
D01new = kbij.get_D01_data('"'+date+'"')

[get_D01_data] Query from BigQuery takes 44.80015230178833


# Data Preparation

In [7]:
print(F01new.shape)
a=[]
for i in F01new.columns:
    a.append(i.replace('_',' '))
F01new.columns = a

(178006, 69)


In [8]:
print(D01new.shape)
a=[]
for i in D01new.columns:
    a.append(i.replace('_',' '))
D01new.columns = a

(126266, 40)


In [9]:
for col in get_key(F01_dtype, 'object'):
    F01new[col] = F01new[col].str.strip()

In [10]:
for col in get_key(D01_dtype, 'object'):
    D01new[col] = D01new[col].str.strip()

In [11]:
F01new = F01new.replace('\n','', regex=True)
D01new = D01new.replace('\n','', regex=True)

In [12]:
F01new = F01new.replace('\t','', regex=True)
D01new = D01new.replace('\t','', regex=True)

In [13]:
D01new.rename(columns={"Alamat email": "Alamat e-mail"}, inplace=True)

In [14]:
gelar_list = pd.read_csv('./data/gelar.csv')['index'].tolist()
gelar_depan_list = pd.read_csv('./data/gelar_depan.csv')['index'].tolist()

In [15]:
def remove_gelar(nama):
    nama_list = nama.split()
    clear_output()
    print(nama_list)
    if nama_list[-1] in gelar_list:
        del nama_list[-1]
    if nama_list[0] in gelar_depan_list:
        del nama_list[0]    
        
    return ' '.join(nama_list)

In [16]:
D01new['Nama Sesuai Identitas'] = D01new['Nama Sesuai Identitas'].apply(remove_gelar)

['ARI', 'ALFIYAN']


In [17]:
D01new['Nama Gadis Ibu Kandung'] = D01new['Nama Gadis Ibu Kandung'].apply(remove_gelar)

['ROSMAWATI']


In [ ]:
F01new.to_csv(path_or_buf=raw_filename_F01, index=False)
D01new.to_csv(path_or_buf=raw_filename_D01, index=False)

In [14]:
DataStorage.upload_notebooks_data(raw_filename_F01, 'kbij/'+pathname, 'f01'+datem+'.csv')

File ./data202002/F01202002.csv version 1 uploaded to notebooks/kbij/data202002/f01202002.csv.


In [15]:
DataStorage.upload_notebooks_data(raw_filename_D01, 'kbij/'+pathname, 'd01'+datem+'.csv')

File ./data202002/D01202002.csv version 1 uploaded to notebooks/kbij/data202002/d01202002.csv.


# Load Data

In [8]:
DataStorage.download_notebooks_data(raw_filename_F01, 'kbij/'+pathname, 'f01'+datem+'.csv')

Blob notebooks/kbij/data201911/f01201911.csv version LATEST downloaded to ./data201911/F01201911.csv.


In [9]:
DataStorage.download_notebooks_data(raw_filename_D01, 'kbij/'+pathname, 'd01'+datem+'.csv')

Blob notebooks/kbij/data201911/d01201911.csv version LATEST downloaded to ./data201911/D01201911.csv.


In [13]:
F01new = pd.read_csv(raw_filename_F01, dtype=F01_dtype)
D01new = pd.read_csv(raw_filename_D01, dtype=D01_dtype)

In [13]:
DataStorage.download_notebooks_data(prev_filtered_filename_F01, 'kbij/'+pathname, 'f01'+datem+'_new_filtered.csv')

Blob notebooks/kbij/data201912/f01201912_new_filtered.csv version LATEST downloaded to ./data201911/F01201911_new_filtered.csv.


In [9]:
DataStorage.download_notebooks_data(prev_filtered_filename_D01, 'kbij/'+pathname, 'd01'+datem+'_new_filtered.csv')

Blob notebooks/kbij/data201911/d01201911.csv version LATEST downloaded to ./data201911/D01201911.csv.


In [6]:
F01_prev = pd.read_csv(prev_filtered_filename_F01, dtype=F01_dtype)
D01_prev = pd.read_csv(prev_filtered_filename_D01, dtype=D01_dtype)

In [21]:
print('D01new:',D01new.shape, 'F01new:',F01new.shape)
print('D01_prev:',D01_prev.shape, 'F01_prev:',F01_prev.shape)

D01new: (126266, 40) F01new: (178006, 69)
D01_prev: (112372, 40) F01_prev: (135874, 69)


# Checking

In [22]:
F01new, D01new = kbij.checking_data(F01new, D01new, date, F01_prev=F01_prev, D01_prev=D01_prev, full=True)

Check Previous Not Active Nomor Rekening Fasilitas But Exist in Current Data 

Found 33423 Nomor Rekening Fasilitas
Drop previous not active Nomor Rekening Fasilitas from current F01
Drop not existing Nomor CIF Debitur in current F01 from current D01

[DONE]
Check Previous Active Nomor Rekening Fasilitas But Not Exist in Current Data 

[CLEAR]No Previous Active Nomor Rekening Fasilitas But Not Exist in Current Data

[DONE]
Check Nomor CIF Debitur in D01 and F01 

[CLEAR]All Nomor CIF Debitur in D01 are exist in F01

[DONE]
Check Active Kode Kondisi but Baki Debet = 0 

Found 52 Nomor Rekening Fasilitas
Set Kode Kondisi to not active

[DONE]
Check Not Active Kode Kondisi but Baki Debet > 0 

[CLEAR]No Not Active Kode Kondisi but Baki Debet > 0

[DONE]
Check Baki Debet > Plafon Awal 

[CLEAR]No Baki Debet greater than Plafon Awal

[DONE]
Check Kode Kualitas Kredit atau Pembiayaan Macet but has no Tanggal Macet 

[CLEAR]No Kode Kualitas Kredit atau Pembiayaan Macet but has no Tanggal Mace

## Check column

In [23]:
F01new[F01new['Denda'].isnull()][['Nomor Rekening Fasilitas','Denda']]

,Nomor Rekening Fasilitas,Denda


In [24]:
F01new[(F01new['Jumlah Hari Tunggakan']==0) & (F01new['Frekuensi Tunggakan']>0)][['orderId','Jumlah Hari Tunggakan','Frekuensi Tunggakan','Approved Date','status']]

,orderId,Jumlah Hari Tunggakan,Frekuensi Tunggakan,Approved Date,status


In [25]:
F01new[(F01new['Tunggakan Pokok']>0) & (F01new['Kode Kualitas Kredit atau Pembiayaan']=='1') & (F01new['Kode Kondisi']=='00')][['orderId','Denda','Kode Kualitas Kredit atau Pembiayaan','Kode Kondisi','dayPastDue']]

,orderId,Denda,Kode Kualitas Kredit atau Pembiayaan,Kode Kondisi,dayPastDue


In [26]:
F01new[(F01new['Denda']==0) & (F01new['Kode Kualitas Kredit atau Pembiayaan']!='1') & (F01new['Kode Kondisi']=='00')][['orderId','Denda','Kode Kualitas Kredit atau Pembiayaan','Kode Kondisi','dayPastDue']]

,orderId,Denda,Kode Kualitas Kredit atau Pembiayaan,Kode Kondisi,dayPastDue
332,ATH-13SVCYFQ,0,5,00,331
449,ATH-14T7HLXJ,0,5,00,230
567,ATH-15KDJSLQ,0,2,00,83
572,ATH-15LJUAP8,0,3,00,96
584,ATH-15PRH8ZH,0,5,00,425
...,...,...,...,...,...
144407,EXT-SGTJREKP,0,4,00,123
144411,EXT-T6USJQCK,0,4,00,180
144493,INS-EJEW9S5N,0,2,00,82
144513,INS-J5QLTQ4R,0,5,00,205


In [27]:
F01new[(F01new['Tunggakan Bunga atau Imbalan']==0) & (F01new['Kode Kualitas Kredit atau Pembiayaan']!='1') & (F01new['Kode Kondisi']=='00')][['orderId','Tunggakan Bunga atau Imbalan','Denda','Kode Kualitas Kredit atau Pembiayaan','Kode Kondisi','dayPastDue']]

,orderId,Tunggakan Bunga atau Imbalan,Denda,Kode Kualitas Kredit atau Pembiayaan,Kode Kondisi,dayPastDue
20018,ATH-5LEAQ3CS,0,661000,5,00,442
25705,ATH-6XQVT373,0,950000,3,00,119
44859,ATH-B9PXR6QK,0,475000,3,00,115
68741,ATH-GTBSMBMB,0,458000,5,00,252
91142,ATH-MW7D4MYP,0,950000,5,00,620
...,...,...,...,...,...,...
144421,EXT-WUNFM9L8,0,950000,2,00,67
144424,EXT-Y7G31EVZ,0,950000,2,00,72
144426,EXT-YAGG8AML,0,750000,2,00,51
144427,EXT-YC15XLHN,0,25000,2,00,4


In [28]:
F01new[(F01new['Jumlah Hari Tunggakan']==0) & (F01new['Kode Kualitas Kredit atau Pembiayaan']=='2') & (F01new['Kode Kondisi']=='00')][['orderId','Denda','Kode Kualitas Kredit atau Pembiayaan','Kode Kondisi']]

,orderId,Denda,Kode Kualitas Kredit atau Pembiayaan,Kode Kondisi


In [29]:
F01new[(F01new['Frekuensi Tunggakan']==0) & (F01new['Kode Kualitas Kredit atau Pembiayaan']=='2') & (F01new['Kode Kondisi']=='00')][['orderId','Denda','Kode Kualitas Kredit atau Pembiayaan','Kode Kondisi']]

,orderId,Denda,Kode Kualitas Kredit atau Pembiayaan,Kode Kondisi


In [30]:
F01new[(F01new['Denda']>0) & (F01new['Kode Kualitas Kredit atau Pembiayaan']!='1') & (F01new['Kode Kondisi']=='02')][['orderId','Denda','Kode Kualitas Kredit atau Pembiayaan','Kode Kondisi']]

,orderId,Denda,Kode Kualitas Kredit atau Pembiayaan,Kode Kondisi


In [31]:
F01new[(F01new['Tunggakan Bunga atau Imbalan']>0) & (F01new['Kode Kualitas Kredit atau Pembiayaan']!='1') & (F01new['Kode Kondisi']=='02')][['orderId','Denda','Kode Kualitas Kredit atau Pembiayaan','Kode Kondisi']]

,orderId,Denda,Kode Kualitas Kredit atau Pembiayaan,Kode Kondisi


# Final File to Send to KBIJ D01 and F01

Null check for CIF and NIK

In [32]:
D01new.loc[(D01new['Nomor CIF Debitur'].isna())|(D01new['Nama Lengkap'].isna())|(D01new['Tanggal Lahir'].isna())|(D01new['Nomor Identitas'].isna())]

,Kode Jenis Pelapor,Kode Pelapor,Tahun Bulan Data,Nomor CIF Lama Debitur,Nomor CIF Debitur,Jenis Identitas,Nomor Identitas,Nama Sesuai Identitas,Nama Lengkap,Kode Status Pendidikan atau Gelar Debitur,...,Kode Kantor Cabang,Operasi Data,Status delete,Create Date,Update Date,Approved Date,userId,orderId,paidoffDate,status


In [33]:
D01new.loc[D01new['userId']=='83528963-c942-4f11-870c-66a424961881', ['Nomor CIF Lama Debitur', 'Nomor CIF Debitur']] = '8177481212127188'
D01new.loc[D01new['userId']=='e8fb52d1-32d8-418f-8dc5-196e6f8e44e8', ['Nomor CIF Lama Debitur', 'Nomor CIF Debitur']] = '8177482213984747'

In [34]:
F01new.loc[F01new['Nomor CIF Debitur'].isna()]

,orderId,Kode Jenis Pelapor,Kode Pelapor,Tahun Bulan Data,Kode Jenis Fasilitas,Nomor Rekening Fasilitas,Nomor Rekening Lama Fasilitas,Nomor CIF Debitur,Kode Sifat Kredit atau Pembiayaan,Kode Jenis Kredit atau Pembiayaan,...,lateFee,balanceWithInterest,balance,dataDate,jumlah cicilan,current payment periode month,count payment,paidoffDate,status,Tunggakan Pokok


In [35]:
F01new.loc[F01new['userId']=='83528963-c942-4f11-870c-66a424961881', ['Nomor CIF Debitur']] = '8177481212127188'
F01new.loc[F01new['userId']=='e8fb52d1-32d8-418f-8dc5-196e6f8e44e8', ['Nomor CIF Debitur']] = '8177482213984747'

## Export full data dump to csv to be used in the next iteration

In [21]:
F01new.to_csv(path_or_buf=filtered_filename_F01, index=0)
D01new.to_csv(path_or_buf=filtered_filename_D01, index=0)

In [16]:
DataStorage.upload_notebooks_data(filtered_filename_F01, 'kbij/'+pathname, 'f01'+datem+'_new_filtered.csv')

File ./data202002/F01202002_new_filtered.csv version 1 uploaded to notebooks/kbij/data202002/f01202002_new_filtered.csv.


In [17]:
DataStorage.upload_notebooks_data(filtered_filename_D01, 'kbij/'+pathname, 'd01'+datem+'_new_filtered.csv')

File ./data202002/D01202002_new_filtered.csv version 1 uploaded to notebooks/kbij/data202002/d01202002_new_filtered.csv.


Export Latest Data

In [38]:
dirname = './data'+datem

In [39]:
timestr = time.strftime("%Y%m%d-%H%M%S")
F01name = dirname+'/F01'+'-'+datem+'['+timestr+']'+'.txt'
D01name = dirname+'/D01'+'-'+datem+'['+timestr+']'+'.txt'
print(F01name,D01name)

./data202002/F01-202002[20200803-160314].txt ./data202002/D01-202002[20200803-160314].txt


In [40]:
F01new_sent=F01new[['Kode Jenis Pelapor', 'Kode Pelapor', 'Tahun Bulan Data',
       'Kode Jenis Fasilitas', 'Nomor Rekening Fasilitas',
       'Nomor Rekening Lama Fasilitas', 'Nomor CIF Debitur',
       'Kode Sifat Kredit atau Pembiayaan',
       'Kode Jenis Kredit atau Pembiayaan',
       'Kode Akad Kredit atau Akad Pembiayaan', 'Nomor Akad Awal',
       'Tanggal Akad Awal', 'Nomor Akad Akhir', 'Tanggal Akad Akhir',
       'Frekuensi Perpanjangan Fasilitas Kredit atau Pembiayaan',
       'Tanggal Awal Kredit atau Pembiayaan', 'Tanggal Mulai',
       'Tanggal Jatuh Tempo', 'Kode Kategori Debitur', 'Kode Jenis Penggunaan',
       'Kode Orientasi Penggunaan', 'Kode Sektor Ekonomi',
       'Kode Kabupaten atau Kota Lokasi Proyek atau Penggunaan Kredit atau Pembiayaan',
       'Nilai Proyek', 'Kode Valuta', 'Suku Bunga atau Imbalan',
       'Jenis Suku Bunga atau Imbalan',
       'Kredit atau Pembiayaan Program Pemerintah', 'Plafon Awal', 'Plafon',
       'Realisasi atau Pencairan Bulan Berjalan', 'Denda', 'Baki Debet',
       'Nilai dalam Mata Uang Asal', 'Kode Kualitas Kredit atau Pembiayaan',
       'Tanggal Macet', 'Kode Sebab Macet', 'Tunggakan Pokok',
       'Tunggakan Bunga atau Imbalan', 'Jumlah Hari Tunggakan',
       'Frekuensi Tunggakan', 'Frekuensi Restrukturisasi',
       'Tanggal Restrukturisasi Awal', 'Tanggal Restrukturisasi Akhir',
       'Kode Cara Restrukturisasi', 'Kode Kondisi', 'Tanggal Kondisi',
       'Keterangan', 'Kode Kantor Cabang', 'Operasi Data', 'Status delete',
       'Create Date', 'Update Date']]
D01new_sent=D01new[['Kode Jenis Pelapor', 'Kode Pelapor', 'Tahun Bulan Data',
       'Nomor CIF Lama Debitur', 'Nomor CIF Debitur', 'Jenis Identitas',
       'Nomor Identitas', 'Nama Sesuai Identitas', 'Nama Lengkap',
       'Kode Status Pendidikan atau Gelar Debitur', 'Jenis Kelamin',
       'Tempat Lahir', 'Tanggal Lahir', 'Nama Gadis Ibu Kandung', 'NPWP',
       'Alamat', 'Kelurahan', 'Kecamatan', 'Kode Kabupaten atau Kota',
       'Kode Pos', 'Nomor Telepon', 'Nomor Telepon Seluler', 'Alamat e-mail',
       'Kode Negara Domisili', 'Kode Pekerjaan', 'Tempat Bekerja',
       'Kode Bidang Usaha Tempat Bekerja', 'Alamat Tempat Bekerja',
       'Kode Golongan Debitur', 'Status Perkawinan Debitur',
       'Kode Kantor Cabang', 'Operasi Data', 'Status delete', 'Create Date',
       'Update Date']]

In [41]:
print(F01new_sent.shape)
print(D01new_sent.shape)

(144583, 53)
(119336, 35)


## Export file for KBIJ Reporting

In [42]:
D01new_sent.to_csv(path_or_buf=D01name, sep='|',index =0, float_format='%.f')
F01new_sent.to_csv(path_or_buf=F01name, sep='|',index =0, float_format='%.f')

In [19]:
DataStorage.upload_notebooks_data(F01name, 'kbij/'+pathname, 'f01-'+datem+'.txt')

File ./data202002/F01-202002[20200803-160314].txt version 1 uploaded to notebooks/kbij/data202002/f01-202002.txt.


In [20]:
DataStorage.upload_notebooks_data(D01name, 'kbij/'+pathname, 'd01-'+datem+'.txt')

File ./data202002/D01-202002[20200803-160314].txt version 1 uploaded to notebooks/kbij/data202002/d01-202002.txt.


# Check Sent Data

In [43]:
F01_sent = pd.read_csv('./data202002/F01-202002[20200803-160314].txt',sep='|',dtype=F01_dtype)
D01_sent = pd.read_csv('./data202002/D01-202002[20200803-160314].txt',sep='|',dtype=D01_dtype)

In [44]:
F01_prev_sent = pd.read_csv('./data202001/F01-202001[20200803-142120].txt',sep='|',dtype=F01_dtype)
D01_prev_sent = pd.read_csv('./data202001/D01-202001[20200803-142120].txt',sep='|',dtype=D01_dtype)

In [45]:
print('D01_sent:',D01_sent.shape, 'F01_sent:',F01_sent.shape)
print('D01_prev_sent:',D01_prev_sent.shape, 'F01_prev_sent:',F01_prev_sent.shape)

D01_sent: (119336, 35) F01_sent: (144583, 53)
D01_prev_sent: (112372, 35) F01_prev_sent: (135874, 53)


In [46]:
F01_sent, D01_sent = kbij.checking_data(F01_sent, D01_sent, date, F01_prev=F01_prev_sent, D01_prev=D01_prev_sent)

Check Previous Not Active Nomor Rekening Fasilitas But Exist in Current Data 

[CLEAR]No Previous Not Active Nomor Rekening Fasilitas But Exist in Current Data

[DONE]
Check Previous Active Nomor Rekening Fasilitas But Not Exist in Current Data 

[CLEAR]No Previous Active Nomor Rekening Fasilitas But Not Exist in Current Data

[DONE]
Check Nomor CIF Debitur in D01 and F01 

[CLEAR]All Nomor CIF Debitur in D01 are exist in F01

[DONE]
Check Active Kode Kondisi but Baki Debet = 0 

[CLEAR]No Active Kode Kondisi but Baki Debet = 0

[DONE]
Check Not Active Kode Kondisi but Baki Debet > 0 

[CLEAR]No Not Active Kode Kondisi but Baki Debet > 0

[DONE]
Check Baki Debet > Plafon Awal 

[CLEAR]No Baki Debet greater than Plafon Awal

[DONE]
Check Kode Kualitas Kredit atau Pembiayaan Macet but has no Tanggal Macet 

[CLEAR]No Kode Kualitas Kredit atau Pembiayaan Macet but has no Tanggal Macet

[DONE]
Check Tahun Bulan Data 

F01: ['202002']
D01: ['202002']

[DONE]
